<a href="https://colab.research.google.com/github/dapopov-st/ExperimentsWithLanguageModels/blob/main/RAG_over_ArXiv_PDFs_Part3_Metadata%2BArXivExplore_single_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'm using [Chris Alexiuk's](https://www.linkedin.com/in/csalexiuk/) [notebook](https://colab.research.google.com/drive/172uMprWwUfEecXQWBrsgDAlkpT_EK39z?usp=sharing)
as a starting point and plan to experiment with some of the ideas from [A Practical Approach to Retrieval Augmented Generation Systems](https://angelinamagr.gumroad.com/l/practical-approach-to-RAG-systems) by Allahyari and Yang.

## Steps
- Make a directory of ArXiv papers on Google Drive
- See if can store the vector store in Drive or just in ram
- If can't access A100 consistenly and run out of GPU memory, make a Mistral 7B/Zephyr 7B version (Zephyr can do ReAct!) -> For now T4 and clean mem is working
- Rag with metadata (ex, subdirectory name etc.). -> For now just including article title with retrieved documents.
- Use Self-Rag 7b? selfrag/selfrag_llama2_7b or 13b.  Think this will help overcome some of the challenges with retrieving from pdfs (getting titles from References section, for example, rather than useful content). UPDATE: fixed this by only including article content up to the References.
- Experiment with different vector stores.  FAISS seems to work for the usecase, but does not allow searching/filtering using metadata.  Consider using Pinecone or Elasticsearch, perhaps.-> Deciding to stick with FAISS and add metadata with article title for now.  

## Get the data and build a Retriever

- Original NB worked in under 10GB on V100

In [1]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"
!pip install -U -q cohere llama-index
!pip install PyPDF2
!pip install pypdf
!pip install -q qdrant-client
!pip install -q -U faiss-cpu tiktoken sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive/')

output_dir = '/content/drive/MyDrive/PdfRag/rag_output_dir'
logging_dir = '/content/drive/MyDrive/PdfRag/rag_logging_dir'
index_dir = '/content/drive/MyDrive/PdfRag/rag_index_dir'

#!ls /content/drive/MyDrive/PdfRag/clusterofstars
%cd /content/drive/MyDrive/PdfRag
#My\ Drive/PdfRag && ls clusterofstars
!ls .

Mounted at /content/drive/
/content/drive/MyDrive/PdfRag
barbie.csv  cache  clusterofstars  instructionmining  oppenheimer.csv  rag_index_dir  testabstract


- The documents consist of a few dozen ArXiv papers about modern LLMs

In [3]:
from pathlib import Path
PDFS_PATH = Path('/content/drive/MyDrive/PdfRag/clusterofstars')
PDFS = list(PDFS_PATH.glob('*.pdf'))
PDFS[0], len(PDFS)

(PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/In-Context Retrieval-Augmented Language Models.pdf'),
 28)

In [7]:
PDFS

[PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/In-Context Retrieval-Augmented Language Models.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Toolformer: Language Models Can Teach Themselves to Use Tools.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Skeleton-of-Thought: Large Language Models Can Do Parallel Decoding.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/SequenceMatch: Imitation Learning for Autoregressive Sequence Modelling with Backtracking.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Chain-of-Thought Prompting Elicits Reasoning in Large Language Models.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Training language models to follow instructions with human feedback.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/LoRA: Low-Rank Adaptation of Large Language Models.pdf'),
 PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/QLORA: Efficient Finetuning of Quantize

In [6]:
# For now, flatten out the file structure for simplicity.  In the future, see if can have Rag use this metadata.
#!mv /content/drive/MyDrive/PdfRag/clusterofstars/ChainOfThought/* /content/drive/MyDrive/PdfRag/clusterofstars

In [4]:
# fastai function to clean GPU memory

import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()



### Task 1: Prepare the data and  build a PDF Data Loader

In [5]:
from PyPDF2 import PdfReader
reader = PdfReader(os.path.expanduser(PDFS[0]))
pages = reader.pages
documents = []
for page in pages:
  documents.append(page.extract_text())
#print(documents[-1])

## First drop everything from References onwards. References were 'confusing' RAG into retrieving primarily titles of papers mentioned there, which is likely not very useful

In [8]:
import PyPDF2

def load_pdf_to_string(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to hold the text
        text = ''

        # Loop through each page and extract the text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            references_index= page_text.upper().find('\nREFERENCES\n')
            if references_index != -1:
              page_text = page_text[:references_index]
              text += page_text
              return text
            text += page_text
    return text

# Use the function to load a PDF into a string
text = load_pdf_to_string(os.path.expanduser(PDFS[1]))

In [21]:
def get_title(pdf_path): return os.path.expanduser(pdf_path).split('/')[-1]

In [23]:
get_title(PDFS[-1])

'Challenges and Applications of Large Language Models.pdf'

In [9]:
text.find('References\n')

-1

In [17]:
PDFS[0]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/In-Context Retrieval-Augmented Language Models.pdf')

In [25]:
all_docs_and_titles = [(load_pdf_to_string(os.path.expanduser(pdf_path)),get_title(pdf_path)) for pdf_path in PDFS]

In [27]:
all_docs = [doc[0] for doc in all_docs_and_titles]
all_titles = [doc[1] for doc in all_docs_and_titles]

In [28]:
len(all_docs)

28

In [29]:
all_docs[-1].find('References\n')

-1

In [30]:
all_docs[0]

'In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the LM. We show that In-Co

In [31]:
all_titles[0]

'In-Context Retrieval-Augmented Language Models.pdf'

dir(PDFS[0])

In [32]:
type(all_docs[0])

str

In [34]:
len(all_docs[0])

37632

In [ ]:
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader(os.path.expanduser(PDFS[0]))
# #pages = loader.load_and_split()
# pages = loader.load()
# all_pages = [PyPDFLoader(os.path.expanduser(PDFS[i])).load() for i in range(len(PDFS))]

- Instead of load_and_split, going for a more custom split to have more control over context length sicnce working with limited compute

In [ ]:
#pages[0], len(pages)

-Either PyPDFLoader or PyPDF2 approach could work

In [ ]:
# from langchain.document_loaders.onedrive_file import CHUNK_SIZE
# from langchain.document_loaders import TextLoader
# from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30

# text_splitter = CharacterTextSplitter(
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap = CHUNK_OVERLAP,
#     length_function=len,
# )
# docs  = text_splitter.split_documents(pages)
# docs

FROM BARBIEHEIMER

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [35]:
from langchain.document_loaders.onedrive_file import CHUNK_SIZE
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.text_splitter import Document

# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 30

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    length_function=len,
)
#text_splitter.split_text(all_pages[0])
# docs = [Document(page_content=pages) for pages in all_pages]
docs  = [text_splitter.split_text(doc) for doc in all_docs]
# # docs

In [36]:
len(docs)

28

In [39]:
tot_len = 0
for text in docs[0]:
    tot_len += len(text)
tot_len #OK, makes sense

37667

In [37]:
docs[0]

['In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the LM. We show that In-C

In [ ]:
PDFS[-1]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Self-Rag.pdf')

In [ ]:
len(docs)

28

In [ ]:
len(docs[0])

62

### Task 2: Create an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

- Not yet sure if should use Qdrant or FAISS


#### Selecting the VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [16]:
from langchain.vectorstores import Qdrant, FAISS

FROM BARBIEHEIMER

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [13]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)


(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [41]:
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents((docs[i][j] for i in range(len(docs)) for j in range(len(docs[i]))), embedder)
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents(docs[0], embedder)
from langchain.schema.document import Document

docs = [Document(page_content=doc[i],metadata={'source':all_titles[j]}) for j,doc in enumerate(docs) for i in range(len(doc))]
for index, pdf in enumerate(docs):
   content = docs[index]
   if index == 0:
       vector_store = FAISS.from_documents([content], embedder)
   else:
      vector_store_i = FAISS.from_documents([content], embedder)
      vector_store.merge_from(vector_store_i)

vector_store
#vector_store.save_local(index_dir)

In [42]:
vector_store.save_local(index_dir)

In [ ]:
# #vector_store = FAISS.from_documents(docs, embedder)
# #vector_store = FAISS.from_documents((docs[i][j] for i in range(len(docs)) for j in range(len(docs[i]))), embedder)
# #vector_store = FAISS.from_documents(docs, embedder)
# #vector_store = FAISS.from_documents(docs[0], embedder)
# from langchain.schema.document import Document

# docs = [Document(page_content=doc[i]) for doc in docs for i in range(len(doc))]
# for index, pdf in enumerate(docs):
#    content = docs[index]
#    if index == 0:
#        vector_store = FAISS.from_documents([content], embedder)
#    else:
#       vector_store_i = FAISS.from_documents([content], embedder)
#       vector_store.merge_from(vector_store_i)


# vector_store.save_local(index_dir)

### To reload the embeddings made above on the next Colab nb use, run the code below.

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.load_local(index_dir, embedder)

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Check that the VectorStore is working by embedding a query and retrieving passages from our reviews that are close to it.

In [ ]:
dir(vector_store)

['_FAISS__add',
 '_FAISS__from',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_aembed_documents',
 '_aembed_query',
 '_asimilarity_search_with_relevance_scores',
 '_cosine_relevance_score_fn',
 '_embed_documents',
 '_embed_query',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_normalize_L2',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_documents',
 'add_embeddings',
 'add_texts',
 'adelete',
 'afrom_documents',
 'afrom_embeddings',
 'afrom_texts',
 'amax_marginal_relevance_search',
 'amax_

In [ ]:
type(vector_store)

langchain.vectorstores.faiss.FAISS

In [ ]:
vector_store.docstore

In [ ]:
# vector = vector_store.reconstruct(0)
# vector_id = vector_store.get_vector_id(vector)

# print(vector)
# print(vector_id)

In [43]:
query = "What is Retrieval-augmented generation?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

provide enough information about whether the produced answer would contradict.
Retrieval-augmented language model pre-training
(REALM) [ 186] inserts retrieved documents
into the pre-training examples. While Guu et al.
[186] designed REALM for extractive tasks
such as question-answering, Lewis et al. [304]
propose retrieval-augmented generation (RAG), a
language generation framework using retrievers
for knowledge-intensive tasks that humans could
not solve without access to an external knowledge
source. Yogatama et al. [646] propose the adaptive
Semiparametric Language Models architecture,
which incorporates the current local context, a
short-term memory that caches earlier-computed
hidden states, and a long-term memory based on a
key-value store of (hidden-state, output) tuples. To
equip a retrieval-augmented LLM with few-shot
abilities that were before only emergent in LLMs
with many more parameters, Izacard et al. [236]
propose a KL-divergence loss term for retrieval
1 I NTRODUCTION

In [44]:
query = "What is Self-Rag?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

predictions are mostly aligned with their assessments. Appendix Table 6 shows several annotated
examples and explanations on assessments.
6 C ONCLUSION
This work introduces SELF-RAG, a new framework to enhance the quality and factuality of LLMs
through retrieval on demand and self-reflection. SELF-RAGtrains an LM to learn to retrieve, generate,
and critique text passages and its own generation by predicting the next tokens from its original
vocabulary as well as newly added special tokens, called reflection tokens. SELF-RAGfurther enables
the tailoring of LM behaviors at test time by leveraging reflection tokens. Our holistic evaluations on
six tasks using multiple metrics demonstrate that SELF-RAGsignificantly outperforms LLMs with
more parameters or with conventional retrieval-augmented generation approaches.
10Preprint.
ETHICAL CONCERNS
This work aims to improve the factuality of LLM outputs, the lack of which continues to cause nu-
3 S ELF-RAG: LEARNING TO RETRIEVE , GENERATE AND C

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [ ]:
%%timeit -n 1 -r 1
query = "What is Self-Rag?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

11.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "What is Self-Rag?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

7.37 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Looked like retrieving from References section may be a problem since references came up frequently for searched terms without being highly informative.  
- REMOVING REFERENCES ONWARDS INCREASED THE RETRIEVAL QUALITY NOTICEABLY
- TODO: Could experiment with different retriever or a smart-enough model (Self-Rag?)

As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we're going to leverage Meta's LLaMA 2!

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`

That's right, a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM.

More information on this model can be found [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)

In [45]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [46]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

(…)a-2-13b-chat-hf/resolve/main/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [47]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-13b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [48]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.9 MB/s eta 0:00:00


In [49]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
) # Get a cudann warning, likely since using T4 vs, say A100

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [50]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [51]:
retriever = vector_store.as_retriever()

In [55]:
from langchain.chains import RetrievalQA,RetrievalQAWithSourcesChain
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)
# qa_with_sources_chain = RetrievalQAWithSourcesChain.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     callbacks=[handler],
#     return_source_documents=True
# )

- Try using RetrievalQAWithSourcesChain

In [54]:
qa_with_sources_chain({"question" : "What makes Self-Rag different from Rag?"})



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


{'question': 'What makes Self-Rag different from Rag?',
 'answer': ' The president did not mention Michael Jackson.\n',
 'sources': '',
 'source_documents': [Document(page_content='3 S ELF-RAG: LEARNING TO RETRIEVE , GENERATE AND CRITIQUE\nWe introduce Self-Reflective Retrieval-Augmented Generation ( SELF-RAG), shown in Figure 1.\nSELF-RAGis a framework that enhances the quality and factuality of an LLM through retrieval and\nself-reflection, without sacrificing LLM’s original creativity and versatility. Our end-to-end training\nlets an LM Mgenerate text informed by retrieved passages, if needed, and criticize the output by\nlearning to generate special tokens. These reflection tokens (Table 1) signal the need for retrieval\nor confirm the output’s relevance, support, or completeness. In contrast, common RAG approaches\nretrieve passages indiscriminately, without ensuring complete support from cited sources.\n3.1 P ROBLEM FORMALIZATION AND OVERVIEW\nFormally, given input x, we train Mt

- Well, so far my use of RetrievalQAWithSourcesChain does not seem to be working well at all!!!  Go back to RetrievalQA

In [57]:
qa_with_sources_chain({"query" : "How does Self-Rag compare to Rag?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How does Self-Rag compare to Rag?',
 'result': " Self-Rag is a new framework that enhances the quality and factuality of LLMs through retrieval and self-reflection, without sacrificing LLM's original creativity and versatility. It is different from conventional RAG approaches that retrieve passages indiscriminately, without ensuring complete support from cited sources. Self-Rag trains an LM to learn to retrieve, generate, and critique text passages and its own generation by predicting the next tokens from its original vocabulary as well as newly added special tokens, called reflection tokens.\n\nPlease let me know if you need any further information or context to answer the question.",
 'source_documents': [Document(page_content='performance. In particular, we randomly sample 5k, 10k, 20k, and 50k instances from our original\n150k training instances, and fine-tune four SELF-RAG 7Bvariants on those subsets. Then, we compare\nthe model performance on PopQA, PubHealth, and ASQA

- However, adding 'source' metadata to Document yielded a result that has has article title, which was one of the desired results.  If end up needing filtering articles by metadata with FAISS, see https://github.com/langchain-ai/langchain/discussions/10983.

In [58]:
qa_with_sources_chain({"query" : "What is QLoRa?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is QLoRa?',
 'result': ' QLoRa is a method for finetuning large language models that uses a combination of 4-bit and 16-bit data types to reduce the required memory for the model. It has been shown to be effective in producing state-of-the-art chatbots that rival ChatGPT.',
 'source_documents': [Document(page_content='and compare QLoRA with 16-bit adapter-finetuning and with full-finetuning for models up to 3B. Our\nevaluations include GLUE [ 58] with RoBERTa-large [ 38], Super-NaturalInstructions (TKInstruct)\n[61] with T5 [ 49], and 5-shot MMLU [ 24] after finetuning LLaMA on Flan v2 [ 39] and Alpaca\n[55]. To additionally study the advantages of NF4 over other 4-bit data types, we use the setup of\nDettmers and Zettlemoyer [13] and measure post-quantization zero-shot accuracy and perplexity\nacross different models (OPT [ 72], LLaMA [ 57], BLOOM [ 52], Pythia [ 7]) for model sizes 125m -\n13B. We provide more details in the results section for each particular setup t

In [ ]:
qa_with_sources_chain({"query" : "Did these papers explore themes of existentialism?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these papers explore themes of existentialism?',
 'result': ' No, these papers do not explore themes of existentialism.',
 'source_documents': [Document(page_content='members and other supporters of the Stanford DAWN project–Facebook, Google, and VMware—as\nwell as the NSF under CAREER grant CNS-1651570. Any opinions, findings, and conclusions or\nrecommendations expressed in this material are those of the authors and do not necessarily reflect\nthe views of the National Science Foundation. Omar Khattab is supported by the Apple Scholars in\nAI/ML fellowship.\n\\usepackage[pdftex]{graphicx} ...\n\\includegraphics[width=0.8\\linewidth]{myfile.pdf}'),
  Document(page_content='members and other supporters of the Stanford DAWN project–Facebook, Google, and VMware—as\nwell as the NSF under CAREER grant CNS-1651570. Any opinions, findings, and conclusions or\nrecommendations expressed in this material are those of the authors and do not necessarily reflect\nthe views of the Na

In [ ]:
qa_with_sources_chain({"query" : " Open large language models (LLMs) with great performance in various tasks have significantly advanced the development of LLMs. However, they are far inferior to commercial models such as ChatGPT and GPT-4 when acting as agents to tackle complex tasks in the real world. These agent tasks employ LLMs as the central controller responsible for planning, memorization, and tool utilization, necessitating both fine-grained prompting methods and robust LLMs to achieve satisfactory performance. Though many prompting methods have been proposed to complete particular agent tasks, there is lack of research focusing on improving the agent capabilities of LLMs themselves without compromising their general abilities. In this work, we present AgentTuning, a simple and general method to enhance the agent abilities of LLMs while maintaining their general LLM capabilities. We construct AgentInstruct, a lightweight instruction-tuning dataset containing high-quality interaction trajectories. We employ a hybrid instruction-tuning strategy by combining AgentInstruct with open-source instructions from general domains. AgentTuning is used to instruction-tune the Llama 2 series, resulting in AgentLM. Our evaluations show that AgentTuning enables LLMs' agent capabilities without compromising general abilities. The AgentLM-70B is comparable to GPT-3.5-turbo on unseen agent tasks, demonstrating generalized agent capabilities. We open source the AgentInstruct and AgentLM-7B, 13B, and 70B models at https://github.com/THUDM/AgentTuning, serving open and powerful alternatives to commercial LLMs for agent tasks. "})

{'query': " Open large language models (LLMs) with great performance in various tasks have significantly advanced the development of LLMs. However, they are far inferior to commercial models such as ChatGPT and GPT-4 when acting as agents to tackle complex tasks in the real world. These agent tasks employ LLMs as the central controller responsible for planning, memorization, and tool utilization, necessitating both fine-grained prompting methods and robust LLMs to achieve satisfactory performance. Though many prompting methods have been proposed to complete particular agent tasks, there is lack of research focusing on improving the agent capabilities of LLMs themselves without compromising their general abilities. In this work, we present AgentTuning, a simple and general method to enhance the agent abilities of LLMs while maintaining their general LLM capabilities. We construct AgentInstruct, a lightweight instruction-tuning dataset containing high-quality interaction trajectories. We

In [104]:
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

In [ ]:
query="Open large language models (LLMs) with great performance in various tasks have significantly advanced the development of LLMs. However, they are far inferior to commercial models such as ChatGPT and GPT-4 when acting as agents to tackle complex tasks in the real world. These agent tasks employ LLMs as the central controller responsible for planning, memorization, and tool utilization, necessitating both fine-grained prompting methods and robust LLMs to achieve satisfactory performance. Though many prompting methods have been proposed to complete particular agent tasks, there is lack of research focusing on improving the agent capabilities of LLMs themselves without compromising their general abilities. In this work, we present AgentTuning, a simple and general method to enhance the agent abilities of LLMs while maintaining their general LLM capabilities. We construct AgentInstruct, a lightweight instruction-tuning dataset containing high-quality interaction trajectories. We employ a hybrid instruction-tuning strategy by combining AgentInstruct with open-source instructions from general domains. AgentTuning is used to instruction-tune the Llama 2 series, resulting in AgentLM. Our evaluations show that AgentTuning enables LLMs' agent capabilities without compromising general abilities. The AgentLM-70B is comparable to GPT-3.5-turbo on unseen agent tasks, demonstrating generalized agent capabilities. We open source the AgentInstruct and AgentLM-7B, 13B, and 70B models at https://github.com/THUDM/AgentTuning, serving open and powerful alternatives to commercial LLMs for agent tasks. "

In [ ]:
os.listdir('.')

['clusterofstars',
 'cache',
 'rag_index_dir',
 'oppenheimer.csv',
 'barbie.csv',
 'testabstract']

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('testabstract').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which papers are most similar to the article with the following summary?  Article summary: " + chain.run(query)

In [ ]:
qa_with_sources_chain({"query" : query})

{'query': "Which papers are most similar to the article with the following summary?  Article summary: \n\nResearchers have developed a method called AgentTuning to enhance the agent capabilities of large language models (LLMs) while maintaining their general abilities. The method uses a hybrid instruction-tuning strategy and a lightweight instruction-tuning dataset called AgentInstruct. The researchers evaluated the method on three LLMs and found that it improved the models' agent capabilities without compromising their general abilities. They also open-sourced the AgentInstruct and AgentLM models to provide open and powerful alternatives to commercial LLMs for agent tasks.",
 'result': ' \n\nThe papers most similar to the article with the given summary are:\n\n1. "AgentTuning: Enhancing Agent Capabilities of Large Language Models" by the same authors. This paper presents the AgentTuning method and its evaluation on three LLMs.\n2. "Tuning Language Models for Agent Tasks" by Ashish Vas

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which papers are most similar to the article with the following summary?  Article summary: " + chain.run(query)

In [ ]:
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'Which papers are most similar to the article with the following summary?  Article summary: \n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.',
 'result': '\n\nBased on the summary, the following papers are most similar to the article:\n\n1. [28] E. J. Hu, Y. Shen, P. Wallis, Z. Allen-Zhu, Y. Li, S. Wang, L. Wang, and W. Chen. Lora: Low-rank adaptation of large language models. arXiv preprint arXi

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which papers are most different to the article with the following summary?  Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'Which papers are most different to the article with the following summary?  Article summary: \n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.',
 'result': '\n\nThe papers that are most different to the article with the given summary are:\n\n1. [28] E. J. Hu, Y. Shen, P. Wallis, Z. Allen-Zhu, Y. Li, S. Wang, L. Wang, and W. Chen. Lora: Low-rank adaptation of large language models. arXiv preprint 

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Which paper has the least in common with the article with the following summary?  Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'Which paper has the least in common with the article with the following summary?  Article summary: \n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.',
 'result': '  The paper with the least in common with the article with the given summary is "Evaluating the harms of language models" because it does not discuss language models or their performance, and instead focuses on the potential negative co

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Be thorough and explain your reasoning step by step.  Which paper has the most in common with the article with the following summary?  .  Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'Be thorough and explain your reasoning step by step.  Which paper has the most in common with the article with the following summary?  .  Article summary: \n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.',
 'result': ' \n\nThe paper with the most in common with the article is "InstructMining: Selecting High-Quality Instruction-Following Data for Large Language Models" by Yi et al. (2019). Both p

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader('instructionmining').load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
query="Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': 'Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: \n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.',
 'result': '  Based on the article summary, the retrieved document that has the most in common with the article is likely to be the one that discusses the proposed method 

In [ ]:
from langchain.document_loaders import TextLoader
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text_list):
        self.text_list = text_list

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        return self.text_list[idx]

# Load the text
query = TextLoader('instructionmining').load()

# Create a dataset
query_dataset = TextDataset(query)

# Run the chain on the dataset
chain.run(query_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'\n\nThis paper proposes a new method called InstructMining for selecting high-quality instruction-following data to optimize the performance of large language models (LLMs) in interacting with humans. The method uses natural language indicators to evaluate the quality of unseen datasets and leverages BlendSearch to find the best subset of the entire dataset. Experiment results show that InstructMining achieves state-of-the-art performance on two popular benchmarks. Additionally, the paper discovers the double descent phenomenon in LLM finetuning and provides insights into the relationship between dataset size and model performance.'

### ArXiv API

In [61]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
locale.getpreferredencoding() ### SOLVED THE UTF-8 ISSUE!

'UTF-8'

In [ ]:
!pip install arxiv

In [ ]:
from langchain.retrievers import ArxivRetrieverM

In [64]:
retriever = ArxivRetriever(load_max_docs=2)

In [66]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 57.5 MB/s eta 0:00:00


In [96]:
docs = retriever.get_relevant_documents(query="2311.05610")

In [121]:
#data=docs[0].metadata  # meta-information of the Documentm

In [120]:
title, summary = docs[0].metadata['Title'],docs[0].metadata['Summary'].replace('\n', ' ')

'Efficient Parallelization Layouts for Large-Scale Distributed Model Training'

In [119]:
import json
with open(title, 'w') as f:
    f.write(summary)

In [87]:
import langchain
from langchain.chains import ConversationalRetrievalChain
from langchain.schema.runnable import Runnable
class MyModelRunnable(Runnable):
    def __init__(self, model):
        self.model = model

    def __call__(self, inputs):
        # This is where you would add the code to run your model on the inputs
        # and return the results. This is just a placeholder example.
        results = self.model.predict(inputs)
        return results
    def invoke(self, inputs):
        return self.__call__(inputs)
qa = ConversationalRetrievalChain.from_llm(MyModelRunnable(model), retriever=retriever)

In [90]:
#help(qa)

In [98]:
query = docs[0].metadata['Summary'].replace('\n', ' ')

In [99]:
query="Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: " + query
qa_with_sources_chain({"query" : query})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: Efficiently training large language models requires parallelizing across hundreds of hardware accelerators and invoking various compute and memory optimizations. When combined, many of these strategies have complex interactions regarding the final training efficiency. Prior work tackling this problem did not have access to the latest set of optimizations, such as FlashAttention or sequence parallelism. In this work, we conduct a comprehensive ablation study of possible training configurations for large language models. We distill this large study into several key recommendations for the most efficient training. For instance, we find that using a micro-batch size of 1 usually enables the most efficient training layouts. Larger micro-batch sizes necessitate activation checkpointing or higher degrees of m

In [100]:
clean_mem()

In [113]:
summary

'Efficiently training large language models requires parallelizing across hundreds of hardware accelerators and invoking various compute and memory optimizations. When combined, many of these strategies have complex interactions regarding the final training efficiency. Prior work tackling this problem did not have access to the latest set of optimizations, such as FlashAttention or sequence parallelism. In this work, we conduct a comprehensive ablation study of possible training configurations for large language models. We distill this large study into several key recommendations for the most efficient training. For instance, we find that using a micro-batch size of 1 usually enables the most efficient training layouts. Larger micro-batch sizes necessitate activation checkpointing or higher degrees of model parallelism and also lead to larger pipeline bubbles. Our most efficient configurations enable us to achieve state-of-the-art training efficiency results over a range of model sizes

In [117]:
#With data saved in JSON
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader(title).load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
#query="Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: " + chain.run(query)
query=f"Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article titled {title} with the following summary? Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article titled Efficient Parallelization Layouts for Large-Scale Distributed Model Training with the following summary? Article summary: \n\nThis paper presents a comprehensive ablation study of possible training configurations for large language models, and distills the results into several key recommendations for the most efficient training. The study finds that using a micro-batch size of 1 usually enables the most efficient training layouts, and that larger micro-batch sizes require activation checkpointing or higher degrees of model parallelism, leading to larger pipeline bubbles. The authors achieve state-of-the-art training efficiency results over a range of model sizes, with a Model FLOPs utilization of 70.5% when training a 13B model.',
 'result': '\n\nThe retrieved documents that have the most in common with the article titled Efficient Paralleliza

In [122]:
#With data saved in JSON
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader(title).load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
#query="Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: " + chain.run(query)
query=f"Be thorough and explain your reasoning step by step.  Which of the retrieved documents with DIFFERENT sources has the most in common with the article titled {title} with the following summary? Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Be thorough and explain your reasoning step by step.  Which of the retrieved documents with DIFFERENT sources has the most in common with the article titled Efficient Parallelization Layouts for Large-Scale Distributed Model Training with the following summary? Article summary: \n\nThe efficient training of large language models requires parallelizing across hardware accelerators and optimizing compute and memory usage. Prior studies did not have access to the latest optimizations, such as FlashAttention and sequence parallelism. This study conducts a comprehensive ablation analysis of training configurations and distills the results into key recommendations for efficient training. The findings include the optimal micro-batch size of 1 and the importance of activation checkpointing and model parallelism for larger models. The study achieves state-of-the-art training efficiency results, with a Model FLOPs utilization of 70.5% for a 13B model.',
 'result': '\n\nThe retrieved d

In [124]:
#With data saved in JSON
chain = load_summarize_chain(llm, chain_type="stuff")
from langchain.document_loaders import TextLoader
query = TextLoader(title).load()
#query = query_loader.load({"text" : "How does Self-Rag compare to Rag?"})
#query="Be thorough and explain your reasoning step by step.  Which of the retrieved documents has the most in common with the article with the following summary?  .  Article summary: " + chain.run(query)
query=f"Be thorough and explain your reasoning step by step.  Which of the retrieved documents have the most in common with the article titled {title} with the following summary? Article summary: " + chain.run(query)
qa_with_sources_chain({"query" : query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


{'query': 'Be thorough and explain your reasoning step by step.  Which of the retrieved documents have the most in common with the article titled Efficient Parallelization Layouts for Large-Scale Distributed Model Training with the following summary? Article summary: \n\nThe efficient training of large language models requires parallelizing across hardware accelerators and optimizing compute and memory usage. Prior studies did not have access to the latest optimizations, such as FlashAttention and sequence parallelism. This study conducts a comprehensive ablation analysis of training configurations and distills the results into key recommendations for efficient training. The findings include the optimal micro-batch size of 1 and the importance of activation checkpointing and model parallelism for larger models. The study achieves state-of-the-art training efficiency results, with a Model FLOPs utilization of 70.5% for a 13B model.',
 'result': '\n\nThe retrieved documents that have the

In [123]:
# questions = [
#     "What are Heat-bath random walks with Markov base?",
#     "What is the ImageBind model?",
#     "How does Compositional Reasoning with Large Language Models works?",
# ]
# chat_history = []

# for question in questions:
#     result = qa({"question": question, "chat_history": chat_history})
#     chat_history.append((question, result["answer"]))
#     print(f"-> **Question**: {question} \n")
#     print(f"**Answer**: {result['answer']} \n")

### Scraping https://paperswithcode.com/area/natural-language-processing.  Set up my own scraper or use LangChain's
- https://python.langchain.com/docs/use_cases/web_scraping